In [1]:
import pandas as pd
data = pd.read_csv("Dane.csv", delimiter=",", header=0)
#print (data)

In [2]:
# data cleaning
data["To"] = data["To"].str.strip()
data["From"] = data["From"].str.strip()
#print (data)

In [3]:
#new links 
for i in range(len(data)):
    data.loc[len(data)] = [data['To'][i], data['From'][i], data['Cost'][i], data['Cap'][i]]
#print (data)

In [4]:
# LabelEncoder -> changes categorical data into numerical
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
labels = data['To'].tolist()    #pone los datos de To en el vector labels
labels = labels + data['From'].tolist() #anade los datos de From en un vector
labels = list(set(labels)) #elimina los datos repetidos
#print (labels)
le.fit(labels)   #indexar nodos
data['From_edge'] = le.transform(data['From'])+1 #guardar indices
data['To_edge'] = le.transform(data['To'])+1     #guardar indices
#print (data['From_edge'])
Vn=0
for i in range(len(data)): 
    if Vn <= data['To_edge'][i]:
        Vn = data['To_edge'][i]
    if Vn <= data['From_edge'][i]:
        Vn = data['From_edge'][i]    
#print (Vn)

In [5]:
data['route_id'] = data.index+1
En=0
for i in range(len(data)): 
    if En <= data['route_id'][i]:
        En = data['route_id'][i]
#print (En)

In [6]:
# add route_id
for i, z in enumerate(list(set((data['From'] + data['To']).tolist()))):
    data.loc[(data['From'] == z[:3]) & (data['To'] == z[3:]), ['route_id']] = i   

In [7]:
data.head(len(data))

,From,To,Cost,Cap,From_edge,To_edge,route_id
0,user1-1,node1,5,6,14,6,1
1,user2-1,node1,5,6,19,6,2
2,user3-1,node1,5,6,24,6,3
3,user4-1,node1,5,6,29,6,4
4,user1-2,node2,5,6,15,7,5
5,user2-2,node2,5,6,20,7,6
6,user3-2,node2,5,6,25,7,7
7,user4-2,node2,5,6,30,7,8
8,user1-3,node3,5,6,16,8,9
9,user2-3,node3,5,6,21,8,10


In [8]:
with open("data.dat", "a") as f:
    f.write("""
data;

param Vn := {Vn};
param En := {En};
param Dn := {Dn};

param : h  s  t :=
 1      {h} {s} {t}
;
        """.format(Vn=Vn, En=En, Dn=1, h=1, s=1, t=5))

In [9]:
with open("data.dat", "a") as f:
    f.write("""
param : A :=
""")

    for i in range(En):       
        f.write("""  {link_number}  {From_edge_number}    1
""".format(link_number=data['route_id'][i], From_edge_number=data['From_edge'][i]))
    f.write(""";""")

In [10]:
with open("data.dat", "a") as f:
    f.write("""

param : B :=
""")
        
    for i in range(En):       
        f.write("""  {link_number}  {to_edge_number}    1
""".format(link_number=data['route_id'][i], to_edge_number=data['To_edge'][i]))
    f.write(""";""")

In [11]:
with open("data.dat", "a") as f:
    f.write("""
     
param : K :=
""")
    for i in range(En):       
        f.write("""  {link_number}  {cost}
""".format(link_number=data['route_id'][i], cost=data['Cost'][i]))

In [12]:
with open("data.dat", "a") as f:
     f.write("""
;
end;
""")